FPL analysis: Who to trade and when?


In [ ]:
In this piece of analysis I will be trying to make the best Weekly predictions for an FPL team selection, with the constraints of 1 free transfer.

I will be using a GitHub repository from Vaastav, who kindly updates with raw data each week, for the past few seasons (https://github.com/vaastav/Fantasy-Premier-League)




In [2]:
##import useful libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import itertools
%matplotlib inline


In [60]:
##url for formatting below 
url = r'https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/{year}/gws/gw{week}.csv'

##set up years/weeks iterator and created a tuple of all combinations
years = ['2016-17','2017-18','2018-19','2019-20']
weeks = np.arange(1,39)


all_combinations = list(itertools.product(years,weeks))

all_combinations[:5]

[('2016-17', 1),
 ('2016-17', 2),
 ('2016-17', 3),
 ('2016-17', 4),
 ('2016-17', 5)]

In [62]:
##create a list of all properly formatted urls 
urls = []

for tup in all_combinations:
    urls.append(url.format(year=tup[0], week = tup[1]))
    
urls[:5]

['https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/gws/gw1.csv',
 'https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/gws/gw2.csv',
 'https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/gws/gw3.csv',
 'https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/gws/gw4.csv',
 'https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/gws/gw5.csv']

In [ ]:

#dfs = []
#for filename in urls:
  #  dfs.append(pd.read_csv(filename,  encoding = 'iso-8859-1', error_bad_lines=False, index=False))

# Concatenate all data into one DataFrame
#main_df = pd.concat(dfs, ignore_index=True)


In [314]:
#its probably easier to import the merged datasets as seperate dataframes

df_2018 =pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/gws/merged_gw.csv', encoding='iso=8859-1', error_bad_lines=False)
df_2019 = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/gws/merged_gw.csv', encoding='iso=8859-1', error_bad_lines=False)
df_2020 = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2020-21/gws/merged_gw.csv', encoding='iso=8859-1', error_bad_lines=False)

In [393]:
player_2019 = pd.read_csv('https://raw.githubusercontent.com/DataJackOH/Fantasy-Premier-League/master/data/2019-20/players_raw.csv', encoding='iso=8859-1', error_bad_lines=False)
player_2020 = pd.read_csv('https://raw.githubusercontent.com/DataJackOH/Fantasy-Premier-League/master/data/2020-21/players_raw.csv', encoding='iso=8859-1', error_bad_lines=False)
player_2019['name']=player_2019['first_name']+player_2019['second_name']
player_2020['name']=player_2020['first_name']+player_2020['second_name']


In [405]:
##fixtures

fixture_2018 = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/fixtures.csv', encoding='iso=8859-1', error_bad_lines=False, parse_dates=['kickoff_time'])
fixture_2019 = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/fixtures.csv', encoding='iso=8859-1', error_bad_lines=False,parse_dates=['kickoff_time'])
fixture_2020 = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2020-21/fixtures.csv', encoding='iso=8859-1', error_bad_lines=False, parse_dates=['kickoff_time'])
fixture_2019

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,stats,team_a,team_a_difficulty,team_a_score,team_h,team_h_difficulty,team_h_score
0,1059702,1,True,True,1,2019-08-09 19:00:00+00:00,90,False,True,"[{'h': [{'value': 1, 'element': 183}, {'value'...",14,5,1,10,2,4
1,1059709,1,True,True,8,2019-08-10 11:30:00+00:00,90,False,True,"[{'h': [], 'a': [{'value': 3, 'element': 214},...",11,2,5,19,4,0
2,1059703,1,True,True,2,2019-08-10 14:00:00+00:00,90,False,True,"[{'h': [{'value': 1, 'element': 61}], 'a': [{'...",15,2,1,3,3,1
3,1059704,1,True,True,3,2019-08-10 14:00:00+00:00,90,False,True,"[{'h': [{'value': 2, 'element': 90}, {'value':...",16,3,0,5,3,3
4,1059705,1,True,True,4,2019-08-10 14:00:00+00:00,90,False,True,"[{'h': [], 'a': [], 'identifier': 'goals_score...",8,3,0,7,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,1060077,47,True,True,376,2020-07-26 15:00:00+00:00,90,False,True,"[{'h': [], 'a': [{'value': 1, 'element': 241},...",12,3,2,9,4,0
376,1060078,47,True,True,377,2020-07-26 15:00:00+00:00,90,False,True,"[{'h': [{'value': 2, 'element': 215}, {'value'...",14,4,0,11,2,5
377,1060079,47,True,True,378,2020-07-26 15:00:00+00:00,90,False,True,"[{'h': [{'value': 1, 'element': 554}], 'a': [{...",10,2,3,13,4,1
378,1060080,47,True,True,379,2020-07-26 15:00:00+00:00,90,False,True,"[{'h': [{'value': 2, 'element': 437}, {'value'...",15,2,1,16,3,3


In [420]:
##team info 
team_2019 = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/teams.csv', encoding='iso=8859-1', error_bad_lines=False)
team_2020 = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2020-21/teams.csv', encoding='iso=8859-1', error_bad_lines=False)


,code,draw,form,id,loss,name,played,points,position,pulse_id,short_name,strength,strength_attack_away,strength_attack_home,strength_defence_away,strength_defence_home,strength_overall_away,strength_overall_home,team_division,unavailable,win
0,3,0,NaN,1,0,Arsenal,0,0,0,1,ARS,4,1170,1170,1200,1150,1240,1180,NaN,False,0
1,7,0,NaN,2,0,Aston Villa,0,0,0,2,AVL,2,980,970,1040,1000,1050,1020,NaN,False,0
2,91,0,NaN,3,0,Bournemouth,0,0,0,127,BOU,2,1030,990,1050,1000,1020,1020,NaN,False,0
3,36,0,NaN,4,0,Brighton,0,0,0,131,BHA,2,1100,1100,1030,1040,1010,1050,NaN,False,0
4,90,0,NaN,5,0,Burnley,0,0,0,43,BUR,3,1070,1130,1110,970,1180,1110,NaN,False,0
5,8,0,NaN,6,0,Chelsea,0,0,0,4,CHE,4,1240,1240,1250,1220,1230,1220,NaN,False,0
6,31,0,NaN,7,0,Crystal Palace,0,0,0,6,CRY,3,1180,1080,1040,1040,1120,1130,NaN,False,0
7,11,0,NaN,8,0,Everton,0,0,0,7,EVE,3,1210,1100,1180,1170,1240,1100,NaN,False,0
8,13,0,NaN,9,0,Leicester,0,0,0,26,LEI,3,1080,1150,1100,1210,1160,1210,NaN,False,0
9,14,0,NaN,10,0,Liverpool,0,0,0,10,LIV,5,1320,1310,1330,1310,1350,1340,NaN,False,0


In [390]:
def playerCleaner(player, team):
    player = player[['team','name']]
    team = team[['strength', 'id', 'strength_attack_home', 'strength_defence_home', 'short_name']]
    team_agg = pd.merge(player, team,  how="inner", left_on = 'team', right_on = 'id')
    return team_agg

In [441]:
def fixtureCleaner(fixture, team):
    fixture = fixture[['finished', 'team_a','team_a_difficulty','team_a_score','team_h','team_h_difficulty','team_h_score']]
    team = team[['id','name' ,'short_name']]
    team_agg = pd.merge(fixture, team,  how="left", left_on = 'team_a', right_on = 'id')
    team_agg = pd.merge(team_agg, team,  how="left", left_on = 'team_h', right_on = 'id', suffixes=('_away','_home'))
    return team_agg

In [192]:
##lets create a function to clean the data
def fplClean(df):
    """this function does 3 things
    1 - cleans non alphabetical charachters from name
    2 - groups by name and gw to create a sensible index
    3 - creates a PPM column, a target for analysis """
    df['name'] = df['name'].str.replace('[^a-zA-Z]', '')
    
    df = df.groupby(['name', 'GW']).mean().reset_index(level=[0,1])
    
    df['PPM'] = df['total_points']/df['value']
    
    return df

In [195]:
##create a form / class attribute (rolling 4 week average for players, 10 week for class)
def formAndClass(df, form_window=4, class_window=10):
    """This function creates form and class columns for analysis. Also adds a 4 game minutes columns
    Default values are 4 for form and 10 weeks for class."""
    df['Form'] = df.groupby(['name']).rolling(form_window)['total_points'].mean().reset_index(level=[0,1],drop=True)
    
    df['Class'] = df.groupby(['name']).rolling(class_window)['total_points'].mean().reset_index(level=[0,1],drop=True)
    return df

In [409]:
def selectColumns(df):
    columns = ['name', 'GW', 'PPM','Form','Class','opponent_team', 'minutes','total_points', 'value', 'was_home']
    df= df[columns]
    return df

In [410]:
##build training set, being careful not to include any data that you'd see after a game
#create a function to find the most up to date rankings for each team where finished = True
def predictionClean(df):
    """creates a target column which is the following weeks score, this will be used to build algorithm"""
    shifted = df.groupby('name').shift(-1)
    shifted = pd.DataFrame(shifted['total_points'])
    df_lag = df.join(shifted.rename(columns=lambda x: "Target"))
    return df_lag

In [411]:
##function to clean players and attach team
player_2019 = playerCleaner(player_2019, team_2019)

In [433]:
##clean each weeks, then join to cleaned player abov

def buildAnalysis(df,player_df, team_df):
    """THIS FUNCTION calls all previous functions, 
    and joins players to there team data, cleans needless columns
    and also adds in the target column"""
    df = fplClean(df)
    df = formAndClass(df)
    df = selectColumns(df)
    df = predictionClean(df)
    df = pd.merge(df, player_df,  how="left", left_on = 'name', right_on = 'name')
    df = pd.merge(df, team_df[['short_name','id','strength']],  how="left", left_on = 'opponent_team', right_on = 'id', suffixes=('_home','_away'))
    return df

In [434]:
df_analysis = buildAnalysis(df_2019,player_2019, team_2019)
df_analysis.head()

,name,GW,PPM,Form,Class,opponent_team,minutes,total_points,value,was_home,Target,team,strength_home,id_home,strength_attack_home,strength_defence_home,short_name_home,short_name_away,id_away,strength_away
0,AaronConnolly,4,0.022222,NaN,NaN,11.0,24.0,1.0,45.0,0.0,1.0,4.0,2.0,4.0,1100.0,1040.0,BHA,MCI,11.0,5.0
1,AaronConnolly,5,0.022222,NaN,NaN,5.0,6.0,1.0,45.0,1.0,1.0,4.0,2.0,4.0,1100.0,1040.0,BHA,BUR,5.0,3.0
2,AaronConnolly,6,0.022222,NaN,NaN,13.0,20.0,1.0,45.0,0.0,1.0,4.0,2.0,4.0,1100.0,1040.0,BHA,NEW,13.0,3.0
3,AaronConnolly,7,0.022222,1.0,NaN,6.0,25.0,1.0,45.0,0.0,13.0,4.0,2.0,4.0,1100.0,1040.0,BHA,CHE,6.0,4.0
4,AaronConnolly,8,0.288889,4.0,NaN,17.0,79.0,13.0,45.0,1.0,1.0,4.0,2.0,4.0,1100.0,1040.0,BHA,TOT,17.0,4.0


In [443]:
##using the fixture cleaner create a cleaned list of fixtures
cleaned_fixture_19 = fixtureCleaner(fixture_2019, team_2019)
cleaned_fixture_19.head()

,finished,team_a,team_a_difficulty,team_a_score,team_h,team_h_difficulty,team_h_score,id_away,name_away,short_name_away,id_home,name_home,short_name_home
0,True,14,5,1,10,2,4,14,Norwich,NOR,10,Liverpool,LIV
1,True,11,2,5,19,4,0,11,Man City,MCI,19,West Ham,WHU
2,True,15,2,1,3,3,1,15,Sheffield Utd,SHU,3,Bournemouth,BOU
3,True,16,3,0,5,3,3,16,Southampton,SOU,5,Burnley,BUR
4,True,8,3,0,7,2,0,8,Everton,EVE,7,Crystal Palace,CRY


In [92]:

columns = ['name', 'assists', 'bonus', 'bps', 'clean_sheets', 'creativity',
       'element', 'fixture', 'goals_conceded', 'goals_scored', 'ict_index',
       'influence', 'kickoff_time', 'minutes', 'opponent_team', 'own_goals',
       'penalties_missed', 'penalties_saved', 'red_cards', 'round', 'saves',
       'selected', 'team_a_score', 'team_h_score', 'threat', 'total_points',
       'transfers_balance', 'transfers_in', 'transfers_out', 'value',
       'was_home', 'yellow_cards', 'GW']


In [102]:
##scale all features 

target = df_2019['PPM']
features = df_2019[['']]

In [414]:
df_2019.tail()

,name,GW,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,PPM,Form,Class
22308,rjanNyland,43,0.0,0.0,0.0,0.0,0.0,27.0,332.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,43.0,0.0,33078.0,3.0,0.0,0.0,0.0,-1161.0,586.0,1747.0,43.0,1.0,0.0,0.0,1.0,0.8
22309,rjanNyland,44,0.0,0.0,0.0,0.0,0.0,27.0,341.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,44.0,0.0,31410.0,0.0,2.0,0.0,0.0,-1673.0,413.0,2086.0,43.0,1.0,0.0,0.0,0.5,0.8
22310,rjanNyland,45,0.0,0.0,0.0,0.0,0.0,27.0,355.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,45.0,0.0,28928.0,1.0,1.0,0.0,0.0,-1438.0,175.0,1613.0,43.0,0.0,0.0,0.0,0.0,0.8
22311,rjanNyland,46,0.0,0.0,0.0,0.0,0.0,27.0,361.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,46.0,0.0,28703.0,0.0,1.0,0.0,0.0,-1224.0,76.0,1300.0,43.0,1.0,0.0,0.0,0.0,0.8
22312,rjanNyland,47,0.0,0.0,0.0,0.0,0.0,27.0,380.0,0.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,47.0,0.0,25958.0,1.0,1.0,0.0,0.0,-1114.0,230.0,1344.0,43.0,0.0,0.0,0.0,0.0,0.8


In [372]:
fixture_2020

,team_a,team_h,team_h_difficulty,team_a_difficulty,kickoff_time,finished,id_away,name_away,short_name_away,id_home,name_home,short_name_home
0,14,2,2,3,NaT,False,2,Aston Villa,AVL,14,Newcastle,NEW
1,14,4,2,2,2021-04-10 14:00:00+00:00,False,4,Burnley,BUR,14,Newcastle,NEW
2,14,7,2,3,2021-01-30 12:30:00+00:00,False,7,Everton,EVE,14,Newcastle,NEW
3,14,17,2,4,2020-09-27 13:00:00+00:00,True,17,Spurs,TOT,14,Newcastle,NEW
4,14,8,2,2,2021-05-23 15:00:00+00:00,False,8,Fulham,FUL,14,Newcastle,NEW
...,...,...,...,...,...,...,...,...,...,...,...,...
375,2,14,3,3,2021-03-13 15:00:00+00:00,False,14,Newcastle,NEW,2,Aston Villa,AVL
376,2,5,3,4,2020-12-28 17:30:00+00:00,True,5,Chelsea,CHE,2,Aston Villa,AVL
377,2,9,3,4,2020-10-18 18:15:00+00:00,True,9,Leicester,LEI,2,Aston Villa,AVL
378,2,20,3,3,2020-12-12 12:30:00+00:00,True,20,Wolves,WOL,2,Aston Villa,AVL
